In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pydantic import BaseModel


In [3]:
PINECONE_API_KEY = "pcsk_5QAmMF_R4DaG9DXTXMYL8fSZmr3GXa7jHZ4jV1stUAbc6T694m2ujSzWBDnnDAeR4eNTEX"
PINECONE_API_ENV = "us-east-1"


In [4]:
import pinecone

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")

In [11]:
#extracted_data

In [12]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 9662


In [14]:
#text_chunks

In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()


C:\Users\Anas\AppData\Local\Temp\ipykernel_2548\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\Users\Anas\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
from pinecone import ServerlessSpec
import os
from pinecone.grpc import PineconeGRPC as Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [5]:

import os
from pinecone.grpc import PineconeGRPC as Pinecone

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [6]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

In [11]:
index_name="medical"

In [12]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)



In [13]:
query = "I have pain in my mouth."

docs=docsearch.similarity_search(query, k=2)

print("Result", docs)

Result [Document(metadata={'page': 977.0, 'source': 'data\\Complete Guide to Prescription  - H. Winter Griffith.pdf'}, page_content='• Dry mouth.\nContinue. Tell doctor at next visit. (Suck ice or\nchew gum).\nInfrequent:\n• Rash, infection, nausea, vomiting.\nDiscontinue. Call doctor right away.\n• Pain in muscles, bones, joints; hair thinning;\ntiredness.\nContinue. Call doctor when convenient.\nRare:\n• Severe stomach pain, bleeding gums, blurred\nvision, severe diarrhea, continuing headache,\nvomiting, eye pain, rectal bleeding, yellow skin\nor eyes, serious depression, psychosis, thoughts\nof suicide.'), Document(metadata={'page': 574.0, 'source': 'data\\Complete Guide to Prescription  - H. Winter Griffith.pdf'}, page_content='Don’t take with:\nAny other medicine for your mouth without consulting your doctor or pharmacist.\n \nOVERDOSE\nSYMPTOMS:')]


In [14]:
# Define medical consultation prompt
prompt_template = """
You are a medical assistant. Use the following medical reference information to answer the patient's question.
If you're unsure or the information isn't in the reference, advise consulting a healthcare professional.

Reference Information: {context}

Patient Question: {question}

Please provide a clear and helpful response, including:
1. Possible causes
2. Suggested solutions
3. When to seek professional medical help

Medical Assistant Response:"""


In [15]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [16]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.2})

In [17]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [18]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\Anas\AppData\Local\Temp\ipykernel_2548\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  
Thank you for asking! Pain in the mouth can have several possible causes, such as dental problems (e.g., cavities, gum disease), oral thrush, or even a sinus infection. It's important to consult with a healthcare professional to determine the underlying cause of your pain and receive proper treatment.

In the meantime, you can try some over-the-counter pain relievers such as ibuprofen or acetaminophen. However, if your symptoms persist or worsen, please seek medical attention immediately. It's always better to be safe than sorry when it comes to oral health!
Response :   Thank you for asking! I'm here to help you with your concern. Based on what you've shared, there could be several reasons why you're experiencing pain in your mouth, including dental problems such as cavities, gum disease, or an abscess. It's important to consult a dentist for proper evaluation and treatment.

In the meantime, here are some things you can try at home:

1. Apply a warm compress to the affec

KeyboardInterrupt: Interrupted by user